# Лабораторная работа 2
Выполнили: Елизарова Юлия, Игошин Андрей, Кипаренко Илья, Кириленко Юлия, Клыков Вячеслав, Лепигина Анастасия

In [42]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler

In [43]:
import sparkmonitor
!jupyter nbextension install sparkmonitor --py --user --symlink 
!jupyter nbextension enable sparkmonitor --py --user            
!jupyter serverextension enable --py --user sparkmonitor
!ipython profile create && echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.py

Installing /Users/stacey/anaconda/lib/python3.6/site-packages/sparkmonitor/static -> sparkmonitor
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable sparkmonitor --user --py
    
Enabling notebook extension sparkmonitor/module...
      - Validating: OK
Enabling: sparkmonitor.serverextension
- Writing config: /Users/stacey/.jupyter
    - Validating...
      sparkmonitor.serverextension  OK


In [44]:
from pyspark import SparkContext
sc=SparkContext.getOrCreate(conf=conf) #Start the spark context

In [45]:
spark = SparkSession.builder.appName('MovieRatingsProject').getOrCreate()

In [46]:
info = spark.read.csv('ml-100k/u.info',inferSchema=True,sep='\t')
info.show()

+--------------+
|           _c0|
+--------------+
|     943 users|
|    1682 items|
|100000 ratings|
+--------------+



Датасет содержит 100 000 оценок 934 пользователями 1682 фильмов. Каждый пользователь оценил не менее 20 фильмов.

In [47]:
data = spark.read.csv('ml-100k/u.data',inferSchema=True,sep='\t')
data = data.withColumnRenamed('_c0','user_id').withColumnRenamed('_c1','movie_id').withColumnRenamed('_c2','rating').withColumnRenamed('_c3','timestamp')
data.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



Информация о каждом фильме включает в себя название, даты выхода в прокат и появления в магазинах, ссылку на IMDB и жанр. Причем один фильм может быть отнесен к нескольким жанрам. 

In [57]:
movies = spark.read.csv('ml-100k/u.item', inferSchema=True, sep='|')
names = ['id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
for i in range(len(names)):
    movies = movies.withColumnRenamed('_c'+str(i), names[i])
movies.show()

+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
| id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+---+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|  1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|  2|    GoldenEye (1995)| 01-Jan-1995|              null|http:/

Информация о каждом пользователе включает в себя возраст, пол, профессию и почтовый индекс.

In [58]:
users = spark.read.csv('ml-100k/u.user', inferSchema=True, sep='|')
names = ['id', 'age', 'gender', 'occupation', 'zip_code']
for i in range(len(names)):
    users = users.withColumnRenamed('_c'+str(i), names[i])
users.show()

+---+---+------+-------------+--------+
| id|age|gender|   occupation|zip_code|
+---+---+------+-------------+--------+
|  1| 24|     M|   technician|   85711|
|  2| 53|     F|        other|   94043|
|  3| 23|     M|       writer|   32067|
|  4| 24|     M|   technician|   43537|
|  5| 33|     F|        other|   15213|
|  6| 42|     M|    executive|   98101|
|  7| 57|     M|administrator|   91344|
|  8| 36|     M|administrator|   05201|
|  9| 29|     M|      student|   01002|
| 10| 53|     M|       lawyer|   90703|
| 11| 39|     F|        other|   30329|
| 12| 28|     F|        other|   06405|
| 13| 47|     M|     educator|   29206|
| 14| 45|     M|    scientist|   55106|
| 15| 49|     F|     educator|   97301|
| 16| 21|     M|entertainment|   10309|
| 17| 30|     M|   programmer|   06355|
| 18| 35|     F|        other|   37212|
| 19| 40|     M|    librarian|   02138|
| 20| 42|     F|    homemaker|   95660|
+---+---+------+-------------+--------+
only showing top 20 rows



Для получения целостного датасета было принято решение собрать датафреймы с оценками и информацией о фильмах и пользователях в один.

In [60]:
joined_df = data.join(users, data.user_id == users.id, 'outer').drop('zip_code', 'timestamp')
joined_df = joined_df.join(movies, joined_df.movie_id == movies.id, 'outer').drop('movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'id')
joined_df.show()

+-------+--------+------+---+------+----------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|movie_id|rating|age|gender|occupation|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+--------+------+---+------+----------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    251|     148|     2| 28|     M|    doctor|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|    580|     148|     4| 16|     M|   student|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0

Категориальные значения (профессия, пол) были оформлены при помощи one-hot encoding.

In [61]:
stringIndexer = StringIndexer(inputCol="gender", outputCol="indexed_gender", handleInvalid='error')
model = stringIndexer.fit(joined_df)
joined_df = model.transform(joined_df)
encoder = OneHotEncoder(inputCol="indexed_gender", outputCol="gender_feature")
joined_df = encoder.transform(joined_df)

stringIndexer = StringIndexer(inputCol="occupation", outputCol="indexed_occupation", handleInvalid='error')
model = stringIndexer.fit(joined_df)
joined_df = model.transform(joined_df)
encoder = OneHotEncoder(inputCol="indexed_occupation", outputCol="occupation_feature")
joined_df = encoder.transform(joined_df)

# mmScaler = MinMaxScaler(inputCol="age", outputCol="scaled_age")
# model = mmScaler.fit(joined_df)
# model.transform(joined_df)

joined_df = joined_df.drop("gender", "indexed_gender", "occupation", "indexed_occupation", "age")
joined_df.show()

+-------+--------+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+--------------+------------------+
|user_id|movie_id|rating|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|gender_feature|occupation_feature|
+-------+--------+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+--------------+------------------+
|    251|     148|     2|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0| (1,[0],[1.0])|   (20,[19],[1.0])|
|    580|     148|     4|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0| 

In [89]:
used_id = data.select('user_id', 'movie_id').collect()
pairs_train = []
for i in range(len(used_id)):
    pairs_train.append((used_id[i].user_id, used_id[i].movie_id))
    
pairs_test = []
for i in range(1, 935):
    for j in range(1, 1683):
        if (i, j) not in pairs_train:
            pairs_test.append((i, j))
            
print(len(pairs_test), len(pairs_train))

KeyboardInterrupt: 

In [92]:
user_345 = joined_df.filter(joined_df.user_id == 345).collect()
user_345

[Row(user_id=345, movie_id=148, rating=3, unknown=0, Action=1, Adventure=1, Animation=0, Children's=0, Comedy=0, Crime=0, Documentary=0, Drama=0, Fantasy=0, Film-Noir=0, Horror=0, Musical=0, Mystery=0, Romance=0, Sci-Fi=0, Thriller=0, War=0, Western=0, gender_feature=SparseVector(1, {}), occupation_feature=SparseVector(20, {7: 1.0})),
 Row(user_id=345, movie_id=471, rating=3, unknown=0, Action=0, Adventure=0, Animation=0, Children's=0, Comedy=0, Crime=0, Documentary=0, Drama=1, Fantasy=0, Film-Noir=0, Horror=0, Musical=0, Mystery=0, Romance=0, Sci-Fi=0, Thriller=0, War=1, Western=0, gender_feature=SparseVector(1, {}), occupation_feature=SparseVector(20, {7: 1.0})),
 Row(user_id=345, movie_id=737, rating=3, unknown=0, Action=0, Adventure=0, Animation=0, Children's=0, Comedy=1, Crime=0, Documentary=0, Drama=1, Fantasy=0, Film-Noir=0, Horror=0, Musical=0, Mystery=0, Romance=0, Sci-Fi=0, Thriller=0, War=0, Western=0, gender_feature=SparseVector(1, {}), occupation_feature=SparseVector(20, {

In [100]:
import numpy as np

genres = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
coefs = {'Action':[], 'Adventure':[], 'Animation':[], "Children's":[], 'Comedy':[], 'Crime':[], 'Documentary':[], 'Drama':[], 'Fantasy':[], 'Film-Noir':[], 'Horror':[], 'Musical':[], 'Mystery':[], 'Romance':[], 'Sci-Fi':[], 'Thriller':[], 'War':[], 'Western':[]}

for i in range(len(user_345)):
    for genre in genres:
        if user_345[i][genre] == 1:
            coefs[genre].append(user_345[i].rating)

for genre in genres:
    coefs[genre] = np.mean(coefs[genre])
print(coefs)

{'Action': 3.5128205128205128, 'Adventure': 3.789473684210526, 'Animation': 2.75, "Children's": 3.625, 'Comedy': 3.642857142857143, 'Crime': 3.7333333333333334, 'Documentary': 5.0, 'Drama': 3.72, 'Fantasy': nan, 'Film-Noir': 5.0, 'Horror': 3.4, 'Musical': 3.7777777777777777, 'Mystery': 3.9, 'Romance': 3.7058823529411766, 'Sci-Fi': 3.6363636363636362, 'Thriller': 3.3617021276595747, 'War': 3.6875, 'Western': 3.25}


/Users/stacey/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/stacey/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [102]:
movie_148 = movies.filter(movies.id == 148).collect()
movie_148

[Row(id=148, movie_title='Ghost and the Darkness, The (1996)', release_date='11-Oct-1996', video_release_date=None, IMDb_URL='http://us.imdb.com/M/title-exact?Ghost%20and%20the%20Darkness,%20The%20(1996)', unknown=0, Action=1, Adventure=1, Animation=0, Children's=0, Comedy=0, Crime=0, Documentary=0, Drama=0, Fantasy=0, Film-Noir=0, Horror=0, Musical=0, Mystery=0, Romance=0, Sci-Fi=0, Thriller=0, War=0, Western=0)]

In [103]:
prediction = []
for genre in genres:
    if movie_148[0][genre] == 1:
        prediction.append(coefs[genre])
print(np.mean(prediction))

3.651147098515519
